In [ ]:
import matplotlib.pyplot as plt 
from matplotlib import rcParams
import seaborn as sns
import pandas as pd 
import numpy as np 
from utils import load_from_pickle, save_to_pickle, mutypes, pcodes
from config import RESULT_PATH, PLOT_PATH
from os import path
from utils import useful_metr

# 1. workflow

# 2/3. balanced model testing on different imbalanced dataset

In [ ]:
from utils import useful_metr
extra_metr = ['BACC', 'F1-score', 'Sensitivity(Recall)', 'Specificity', 'Precision', 'AUPRC', 'OFO']

In [ ]:
data2_df_list = []
software = 'gatk'
for mutype in mutypes:
    for pcode in pcodes:
        respath = path.join(RESULT_PATH, software, mutype, pcode)
        for imbr_test in [1, 50, 100, 200, 0]:
            _, metrs_df = load_from_pickle(path.join('metr', 'metrs_0_30x_{:.1f}-{:.1f}.pkl'.format(1,imbr_test)), respath)
            marker_dict = {
                'seed': 0, 
                'mutation':mutype, 
                'patient':pcode, 
                'ratio train':1,
                'ratio test':imbr_test,
                'depth': '30X'}
            used_metr_df = useful_metr(metrs_df[0], marker_dict, extra_metr)
            data2_df_list.append(used_metr_df)
data2_df = save_to_pickle(pd.concat(data2_df_list), 'data2_BTrImbTe.pickle', PLOT_PATH)
# data2_df


# 4. rebalanced models testing on imbalanced dataset

In [ ]:
software = 'gatk'
data4_df_list = []
for mutype in mutypes:
    for pcode in pcodes:
        respath = path.join(RESULT_PATH, software, mutype, pcode)
        for reb_code in ['0.0-0.0', 'balanced', 'smote', 'randomover', 'nearmiss1', '1.0-0.0']:
            _, metrs_df = load_from_pickle(path.join('metr', 'metrs_0_30x_{}.pkl'.format(reb_code)), respath)
            marker_dict = {
                'seed': 0, 
                'mutation':mutype, 
                'patient':pcode, 
                'balance strategy':reb_code,
                'ratio test':0,
                'depth': '30X'}
            used_metr_df = useful_metr(metrs_df[0], marker_dict, ['F1-score', 'AUPRC','OFO'])
            data4_df_list.append(used_metr_df)
data4_df = save_to_pickle(pd.concat(data3_df_list), 'data4_RebTrImbTe.pickle', PLOT_PATH)
data4_df


# 5. ROC and decision points for diafferent filters and different callers

In [ ]:
from sklearn.metrics import roc_curve, auc
caller_alias = {'gatk':'gatk','mutect2':'mutect2','varscan':'varscan2'}
filter_alias = {'garfield':'Gar', 'tc':'FNVC', 'vqsr':'VQSR'}
YSCORE_PATH = '/data1/TumorGroup/DATA/public_database/FVC/Data/third_check/predict_result'
PLOT_PATH = '/export/home/zhouxiaocheng/project/test/ryy/plot'
sec = 101
mean_fpr = np.linspace(0,1,sec+1)

In [ ]:
depth = '50X' # change it to '30X'
data5_df_list = []
for caller in ['gatk','mutect2','varscan']:
    metr_df = load_from_pickle('data4_{caller}CV.pickle'.format(caller=caller), PLOT_PATH)
    metr_df['depth'] = metr_df['depth'].apply(lambda x: x.upper())
    for mutation in ['snp', 'indel']:
        for pcode in ['HG00'+str(i) for i in [1,2,3,4,6,7]]:
            score_fn = '{pcode}_{caller}_predict_results_all_{mutation}_score.txt'.format(pcode=pcode,caller=caller_alias[caller],mutation=mutation)
            score_df = pd.read_csv(path.join(YSCORE_PATH,depth,pcode,score_fn), sep='\t')
            for filter, af in filter_alias.items():
                col_af = [col for col in score_df.columns if af in col][0]
                pre_metr = metr_df.loc[(metr_df['mutation']==mutation)&
                                       (metr_df['patient']==pcode)&
                                       (metr_df['filter']==filter)&
                                       (metr_df['depth']==depth)].iloc[0].to_dict()
                fpr, tpr, _ = roc_curve(score_df['y_true'].values, score_df[col_af].values, pos_label=1)
                if np.abs(pre_metr['AUC'] - auc(fpr,tpr)) > 1e-5:
                    print('BIG', pre_metr['AUC'] - auc(fpr,tpr), filter,'########### WARNING!!! ########## {}'.format(score_fn))
                mean_tpr = np.interp(mean_fpr, fpr, tpr)
                mean_tpr[0] = 0.
                mean_tpr[sec] = 1.
                if np.abs(pre_metr['AUC'] - auc(mean_fpr,mean_tpr)) > 1e-5:
                    print('SMALL', pre_metr['AUC'] - auc(mean_fpr,mean_tpr), filter,'########### WARNING!!! ########## {}'.format(score_fn))
                data5_df_list.append(pd.DataFrame({'fpr':mean_fpr,'tpr':mean_tpr,'caller':caller,'mutation':mutation,'pcode':pcode,'filter':filter}))
    #         pcode_pack[pcode] = filter_pack
    #     mutation_pack[mutation] = pcode_pack
    # caller_pack[caller] = mutation_pack
data5_df = pd.concat(data5_df_list)
_ = save_to_pickle(data5_df, 'data5_forauc_{}_sparse.pickle'.format(depth), PLOT_PATH)

# 6. extra filter methods and our models based on the feature from different callers

In [ ]:
from config import DATABASE_PATH, DATABASE_TWO_PATH
from utils import gbeta_, fbeta_
def get_metr(df, metrname):
    try:
        return df[metrname].values
    except:
        return [None]

def load_statistic(caller, mutype, pcode):
    if caller == 'varscan':
        caller += '2'
    else:
        depthlist = ['30X', '50X']
    new_metrs_df_list = []
    for depth in depthlist:
        for filter in ['freq', 'garfield', 'hf', 'vqsr']:
            filepath = path.join(DATABASE_PATH, depth, pcode, '{}_{}_{}_statistic.txt'.format(caller, filter, mutype))
            try:
                metr_df = pd.read_csv(filepath, sep='\t')
                MCC = metr_df['MCC'].values[0]
                OFO = metr_df['FN'].values[0] / metr_df['TN'].values[0]
                gb_dict = metr_df[['TP', 'FP', 'TN', 'FN']].T.to_dict()[0]
                G1 = gbeta_(**gb_dict)
                F1_calcul = fbeta_(**gb_dict)
                ACC = (gb_dict['TP']+gb_dict['TN']) / (gb_dict['TP']+gb_dict['FN']+gb_dict['TN']+gb_dict['FP'])
                TPR = gb_dict['TP'] / (gb_dict['TP']+gb_dict['FN'])
                TNR = gb_dict['TN'] / (gb_dict['TN']+gb_dict['FP'])
                PPV = gb_dict['TP'] / (gb_dict['TP']+gb_dict['FP'])
                BACC = 0.5 * (TPR + TNR)
                AUC = get_metr(metr_df, 'AUC')[0]
                F1_infile = get_metr(metr_df, 'F1')[0]
                AUPRC = get_metr(metr_df, 'AUPRC')[0]
                if F1_infile != None:
                    if np.abs(F1_infile - F1_calcul) > 0.0001:
                        print('### Warning!!! inconsistant F1-score! '*3)
                new_metr_df = pd.DataFrame({
                    'AUC':[AUC], 'MCC':[MCC], 'BACC':[BACC], 'F1-score':[F1_calcul],
                    'Sensitivity(Recall)':[TPR], 'Specificity':[TNR], 'ACC':[ACC],
                    'Precision':[PPV], 'AUPRC':[AUPRC], 'OFO':[OFO], 'G1-score':[G1],
                    'TP':gb_dict['TP'], 'FP':gb_dict['FP'], 'TN':gb_dict['TN'], 'FN':gb_dict['FN'],
                    'depth':[depth], 'filter':[filter],})
                new_metrs_df_list.append(new_metr_df)
            except Exception as e:
                print(e)
    new_metrs_df = pd.concat(new_metrs_df_list)
    new_metrs_df['seed'] = None
    new_metrs_df['caller'] = caller
    new_metrs_df['mutation'] = mutype
    new_metrs_df['patient'] = pcode
    new_metrs_df['ratio test'] = 0
    return new_metrs_df


In [ ]:
extra_metr = ['ACC','BACC', 'F1-score', 'Sensitivity(Recall)', 'Specificity', 'Precision', 'AUPRC', 'OFO','TP','FN','TN','FP']

In [ ]:
for caller in ['gatk', 'mutect2', 'varscan']:
    data6_df_list = []
    for mutype in mutypes:
        for pcode in pcodes:
            # load model metrs
            respath = path.join(RESULT_PATH, caller, mutype, pcode)
            for feat in ['tc']:
                _, metrs_df = load_from_pickle(path.join('metr', 'metrs_0_{}.pkl'.format(feat)), respath)
                marker_dict = {
                    'seed': 0, 
                    'caller': caller,
                    'mutation':mutype, 
                    'patient':pcode, 
                    'filter':feat,
                    'ratio test':0}
                used_metrs_df = [useful_metr(metr_df, marker_dict, extra_metr) for metr_df in metrs_df]
                used_metrs_df[0]['depth'] = '30X'
                used_metrs_df[1]['depth'] = '50x'
                data6_df_list.append(pd.concat(used_metrs_df))
            
            # load existed filters metrs
            data6_df_list.append(load_statistic(caller, mutype, pcode))
    data6_df = save_to_pickle(pd.concat(data6_df_list), 'data6_{}CV.pickle'.format(caller), PLOT_PATH)


# prepare for 7/8/9. load y* and additional info from record


In [ ]:
def readRecord(file_path1, file_path2):
    data1 = pd.read_csv(file_path1, header=None, sep=' ', low_memory=False)
    data2 = pd.read_csv(file_path2, header=None, sep=' ', low_memory=False)
    
    record_df = pd.concat([data1, data2], ignore_index=True).iloc[:, [0,1,2,4,5]]
    record_df.columns = ['y_check', 'chr', 'loc', 'pre_mut', 'post_mut']
    return record_df

def load_record(pcode, mutype='indel', caller='gatk', depth='30X', feat='tc'):
    
    if feat == 'tc':
        root_path = DATABASE_PATH
    elif feat == 'gf':
        root_path = DATABASE_TWO_PATH

    record_df = readRecord(
        path.join(root_path, depth, pcode, '{}_{}_fp_{}_feature.record'.format(pcode, caller, mutype)),
        path.join(root_path, depth, pcode, '{}_{}_tp_{}_feature.record'.format(pcode, caller, mutype)))
    return record_df

In [ ]:
depthlist = ['30X', '50X']
for caller in ['gatk', 'varscan', 'mutect2']:
    for mutype in mutypes:
        for pcode in pcodes:
            respath = path.join(RESULT_PATH, caller, mutype, pcode)
            y_df_tc, _ = load_from_pickle(path.join('metr', 'metrs_0_tc.pkl'), respath)
            records_df = [load_record(pcode, mutype, caller, depth) for depth in depthlist]

            for i, depth in enumerate(depthlist):
                y_df_tc[i].columns = ['y_true', 'y_pred_tc', 'y_score_tc']
                if (records_df[i]['y_check'].values == y_df_tc[i]['y_true'].values).mean() != 1:
                    print('### Error!!! inconsistant y_true in {}-th element in {}/{}/{}'.format(i, pcode, caller, mutype))
                record_full = pd.concat((records_df[i], y_df_tc[i][['y_pred_tc', 'y_score_tc']]), axis=1)
                record_full.to_csv(path.join(PLOT_PATH, 'record_y/{}/{}/{}_{}_{}.txt'.format(depth, pcode, pcode, caller, mutype)), sep='\t', index=False)


# 7. group by frequency

In [ ]:
data7_df_list_less = []
data7_df_list_greater = []
for caller in ['gatk', 'mutect2', 'varscan2']:
    for depth in depthlist:
        for pcode in pcodes:
            df = pd.concat([pd.read_csv('/data1/TumorGroup/DATA/public_database/FVC/Data/third_check/predict_result/{}/{}/{}_{}_predict_results_{}_all_with_cds.txt'.format(depth,pcode,pcode,caller,consis), sep='\t', low_memory=False) for consis in ['consistent', 'inconsistent']])
            less_df = consis_metr(df.loc[df['Frequency']<0.2], needed_metr)
            grea_df = consis_metr(df.loc[df['Frequency']>=0.2], needed_metr)

            less_df['caller'] = caller
            less_df['depth'] = depth
            less_df['pcode'] = pcode 
            less_df['coding'] = None
            grea_df['caller'] = caller
            grea_df['depth'] = depth
            grea_df['pcode'] = pcode
            grea_df['coding'] = None
            data7_df_list_less.append(less_df)
            data7_df_list_greater.append(grea_df)
data7_less_metr_df = save_to_pickle(pd.concat(data7_df_list_less), 'data7_less.pickle', PLOT_PATH)
greater_metr_df = save_to_pickle(pd.concat(data7_df_list_greater), 'data7_greater.pickle', PLOT_PATH)

# 8/9. report inconsistant and consistent result (hard detect and easy detect)

In [ ]:
from utils import evaluator_block
from sklearn.metrics import confusion_matrix

In [ ]:
def consis_metr(df, m=None):
    if m is None:
        m = ['ACC','Sensitivity(Recall)','FDR','Specificity','Precision','NPV','MCC', 'filter']
    df = df.loc[df.y_true != 'y_true']
    y_true = df.y_true.values.astype(int)
    cols_pred = [col for col in df.columns if 'red' in col]
    metr_dict = {}
    for col in cols_pred:
        metr = evaluator_block(y_true, df[col].values.astype(int), df[col].values.astype(int))
        metr['filter'] = col.split('_')[0]
        metr_dict[col.split('_')[0]] = metr
    return pd.DataFrame(metr_dict).loc[m].T


In [ ]:
needed_metr = ['MCC','BACC','ACC','F1-score','G1-score','Specificity','Sensitivity(Recall)','OFO','FDR','Precision','NPV','TP','FP','TN','FN','filter']
depthlist = ['30X','50X']

In [ ]:
for consis in ['consistent', 'inconsistent']:
    data8_df_list = []
    for caller in ['gatk', 'mutect2', 'varscan2']:
        for depth in depthlist:
            for pcode in pcodes:
                df = pd.read_csv('/data1/TumorGroup/DATA/public_database/FVC/Data/third_check/predict_result/{}/{}/{}_{}_predict_results_{}_all_with_cds.txt'.format(depth,pcode,pcode,caller,consis), sep='\t', low_memory=False)
                nonc_df = consis_metr(df.loc[df['Coding']==0], needed_metr)
                code_df = consis_metr(df.loc[df['Coding']==1], needed_metr)

                nonc_df['caller'] = caller
                nonc_df['depth'] = depth
                nonc_df['pcode'] = pcode 
                nonc_df['coding'] = 0
                code_df['caller'] = caller
                code_df['depth'] = depth
                code_df['pcode'] = pcode
                code_df['coding'] = 1
                data8_df_list.append(nonc_df)
                data8_df_list.append(code_df)
    consis_metr_df = save_to_pickle(pd.concat(df_list), 'data8_{}.pickle'.format(consis), PLOT_PATH)
            

# Supp 1. load importance of feature

In [ ]:
caller = 'gatk'
mutation = 'snp'
pcode = 'HG001'
clfpath = path.join(RESULT_PATH, caller, mutation, pcode, 'clf')
clfkit, _ = load_from_pickle('clfkits_0_tc.pkl',clfpath)

In [ ]:
importance = pd.DataFrame(clfkit[0]['xgbdef'].feature_importances_.reshape(1,-1))
importance[['caller','mutation','pcode']] = [[caller, mutation, pcode]]


In [ ]:
for caller in ['gatk','mutect2','varscan']:
    importance_list = []
    for mutation in ['snp','indel']:
        for pcode in ['HG001','HG002','HG003','HG004','HG006','HG007']:
            clfpath = path.join(RESULT_PATH, caller, mutation, pcode, 'clf')
            clfkit, _ = load_from_pickle('clfkits_0_tc.pkl',clfpath)
            importance = pd.DataFrame(clfkit[0]['xgbdef'].feature_importances_.reshape(1,-1))
            importance[['caller','mutation','pcode']] = [[caller, mutation, pcode]]
            importance_list.append(importance)
    imp_df = save_to_pickle(pd.concat(importance_list), 'dataS1_imp_{}.pickle'.format(caller), PLOT_PATH)
